## <div align="center"> UJIAN TENGAH SEMESTER IF540-L MACHINE LEARNING </div>
## <div align="center"> Semester Gasal 2024/2025 </div>
## <div align="center"> Pengembangan Model Prediksi Tipe Curah Hujan dengan Analisis Regresi pada Data Cuaca Historis </div>

---
### Kelompok - 4

#### Anggota Kelompok : 
| Nama                          | NIM         |
| ----------------------------- | ----------- |
| Brightly Virya                | 00000068227 |
| Batara Hotdo Horas Simbolon   | 00000078626 |
| Daniel Isaach Hatuaon Siregar | 00000078609 |
| Christopher Alvaro            | 00000073214 |

---

### Dataset yang digunakan untuk projek:

1. weatherHistory.csv – sumber : https://www.kaggle.com/datasets/muthuj7/weather-dataset

### Instaling dependencies
* don't need to run this code if you already have all the dependencies

In [66]:
%pip install pandas
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a las

### Import all the libraries

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

### Pre-processing

### Load train data

In [68]:
df = pd.read_csv('WeatherHistory.csv')
df

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


#### Handle missing values di kolom 'Precip Type' dengan di isi most frequent value

In [69]:
precip_imputer = SimpleImputer(strategy='most_frequent')
df['Precip Type'] = precip_imputer.fit_transform(df[['Precip Type']]).ravel()

### Drop kolom yang tidak di butuhkan untuk logistic regression
#### * 'Formatted Date', 'Summary', dan 'Daily Summary' di drop karena tidak cocok dengan logistic regression karena non-numeric.
#### * 'Formated Date' di format ulang karena tidak bisa digunakan langsung pada logistic regression

In [70]:
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)
df['Year'] = df['Formatted Date'].dt.year
df['Month'] = df['Formatted Date'].dt.month
df['Day'] = df['Formatted Date'].dt.day
df['Hour'] = df['Formatted Date'].dt.hour
df = df.drop(columns=['Formatted Date'])

df = df.drop(columns=['Summary', 'Daily Summary'])

### Encode categorical data di kolom 'Precip Type' jadi numerical

In [71]:

label_encoder = LabelEncoder()
df['Precip Type'] = label_encoder.fit_transform(df['Precip Type'])
df

,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Year,Month,Day,Hour
0,0,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,2006,3,31,22
1,0,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,2006,3,31,23
2,0,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,2006,4,1,0
3,0,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,2006,4,1,1
4,0,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,2006,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96448,0,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,2016,9,9,17
96449,0,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,2016,9,9,18
96450,0,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,2016,9,9,19
96451,0,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,2016,9,9,20


### pastikan semua feature columns numeric dan handle segala data yang non numeric
* Fungsi pd.get_dummies() digunakan untuk mengubah data kategorikal menjadi beberapa kolom biner (0 atau 1) sehingga model dapat memprosesnya dengan baik. Dengan drop_first=True, kita menghindari masalah multikolinearitas. dengan menghapus satu kategori sebagai referensi.

In [72]:
df = pd.get_dummies(df, drop_first=True)
print(df)

       Precip Type  Temperature (C)  Apparent Temperature (C)  Humidity  \
0                0         9.472222                  7.388889      0.89   
1                0         9.355556                  7.227778      0.86   
2                0         9.377778                  9.377778      0.89   
3                0         8.288889                  5.944444      0.83   
4                0         8.755556                  6.977778      0.83   
...            ...              ...                       ...       ...   
96448            0        26.016667                 26.016667      0.43   
96449            0        24.583333                 24.583333      0.48   
96450            0        22.038889                 22.038889      0.56   
96451            0        21.522222                 21.522222      0.60   
96452            0        20.438889                 20.438889      0.61   

       Wind Speed (km/h)  Wind Bearing (degrees)  Visibility (km)  Loud Cover  \
0                1

### Split fitur dan variabel target
#### * X adalah kolom temperature, apparent temperature, humidity, dan windspeed
#### * Y nya adalah pecip type

In [73]:
X = df.drop(columns=['Precip Type'])
y = df['Precip Type']
print(X)
print(y)

       Temperature (C)  Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0             9.472222                  7.388889      0.89            14.1197   
1             9.355556                  7.227778      0.86            14.2646   
2             9.377778                  9.377778      0.89             3.9284   
3             8.288889                  5.944444      0.83            14.1036   
4             8.755556                  6.977778      0.83            11.0446   
...                ...                       ...       ...                ...   
96448        26.016667                 26.016667      0.43            10.9963   
96449        24.583333                 24.583333      0.48            10.0947   
96450        22.038889                 22.038889      0.56             8.9838   
96451        21.522222                 21.522222      0.60            10.5294   
96452        20.438889                 20.438889      0.61             5.8765   

       Wind Bearing (degree

### Standardize the features
* Standarisasi mengubah fitur sehingga memiliki rata-rata 0 dan deviasi standar 1.
* Ini memastikan semua fitur memberikan kontribusi yang setara pada model, sehingga fitur dengan skala yang lebih besar tidak mendominasi.
* X_standar = (X - mean) / std_dev
* Di mana 'mean' adalah rata-rata dari fitur dan 'std_dev' adalah standar deviasi dari fitur.

In [74]:
columns_to_scale = X.columns.difference(['Year', 'Month', 'Day', 'Hour'])
scaler = StandardScaler()
X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])
X

,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Year,Month,Day,Hour
0,-0.257599,-0.324035,0.793470,0.478635,0.591256,1.306976,0.0,0.101685,2006,3,31,22
1,-0.269814,-0.339097,0.639996,0.499594,0.665756,1.306976,0.0,0.105960,2006,3,31,23
2,-0.267487,-0.138102,0.793470,-0.995473,0.153570,1.099586,0.0,0.108610,2006,4,1,0
3,-0.381489,-0.459071,0.486521,0.476306,0.758881,1.306976,0.0,0.112628,2006,4,1,1
4,-0.332631,-0.362469,0.486521,0.033841,0.665756,1.306976,0.0,0.113483,2006,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,1.474532,1.417400,-1.559811,0.026855,-1.457488,1.372265,0.0,0.095102,2016,9,9,17
96449,1.324468,1.283404,-1.304020,-0.103556,-1.559925,1.241686,0.0,0.101942,2016,9,9,18
96450,1.058076,1.045534,-0.894753,-0.264241,-1.466800,1.372265,0.0,0.106216,2016,9,9,19
96451,1.003983,0.997233,-0.690120,-0.040680,-1.559925,1.372265,0.0,0.108696,2016,9,9,20


### Split the data into training and testing sets

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Features after preprocessing:", X[:5])
print("Target after preprocessing:", y[:5])

Features after preprocessing:    Temperature (C)  Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0        -0.257599                 -0.324035  0.793470           0.478635   
1        -0.269814                 -0.339097  0.639996           0.499594   
2        -0.267487                 -0.138102  0.793470          -0.995473   
3        -0.381489                 -0.459071  0.486521           0.476306   
4        -0.332631                 -0.362469  0.486521           0.033841   

   Wind Bearing (degrees)  Visibility (km)  Loud Cover  Pressure (millibars)  \
0                0.591256         1.306976         0.0              0.101685   
1                0.665756         1.306976         0.0              0.105960   
2                0.153570         1.099586         0.0              0.108610   
3                0.758881         1.306976         0.0              0.112628   
4                0.665756         1.306976         0.0              0.113483   

   Year  Month  Day  Hour 

### Menyimpan data setelah praproses ke dalam file CSV

In [76]:
processed_df = pd.DataFrame(X, columns=df.drop(columns=['Precip Type']).columns)
processed_df['Precip Type'] = y.reset_index(drop=True)
processed_df.to_csv('processed_weather_data.csv', index=False)

### Hasil kerja model logistic regression

In [77]:
# Your codes are here (replace the following codes)
import time
time.sleep(10)

In [78]:
!jupyter nbconvert --to html "./UTS2024_IF540L_E_Kelompok_4.ipynb" --output-dir="./"

[NbConvertApp] Converting notebook ./UTS2024_IF540L_E_Kelompok_4.ipynb to html
[NbConvertApp] Writing 323891 bytes to UTS2024_IF540L_E_Kelompok_4.html


### Next step:
* convert the generated html file to PDF using the online tool: https://www.sejda.com/html-to-pdf
* choose the following settings:
    * Page size: One long page
    * Page Orientation: auto
    * Use print stylesheet
* Submit your ipython notebook and PDF files

Markdown basics https://markdown-guide.readthedocs.io/en/latest/basics.html#